In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import xticks
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow as tf
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,Activation,Dropout
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,LSTM

Using TensorFlow backend.


We can see a lots of null values for "keyword" and "location" columns

In [2]:
# lets use only tweet text to build the model
X = train.text
y = train.target

test_id = test.id
test.drop(["id","location","keyword"],1,inplace = True)

NameError: name 'train' is not defined

# Code Implementation in Tensorflow 2.0

In [3]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

# Loading input files, BERTS seems to do better on raw data

In [5]:
# load train and test datasets
train= pd.read_csv('../input/data-set/train.csv')
test=pd.read_csv('../input/data-set/test.csv')

In [6]:
# check the no. of rows and columns in the dataset
train.shape, test.shape

((16962, 3), (5654, 2))

In [7]:
train.isnull().sum().sort_values(ascending = False)

target    0
text      0
id        0
dtype: int64

In [8]:
### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [9]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 1min 33s, sys: 11.4 s, total: 1min 44s
Wall time: 1min 50s


In [11]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [12]:
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)


In [13]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [14]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [15]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

Train on 13569 samples, validate on 3393 samples
Epoch 1/3
13569/13569 [==============================] - 884s 65ms/sample - loss: 0.3945 - accuracy: 0.8324 - val_loss: 0.3618 - val_accuracy: 0.8512
Epoch 2/3
13569/13569 [==============================] - 835s 62ms/sample - loss: 0.2989 - accuracy: 0.8804 - val_loss: 0.3513 - val_accuracy: 0.8541
Epoch 3/3
13569/13569 [==============================] - 835s 62ms/sample - loss: 0.2218 - accuracy: 0.9161 - val_loss: 0.3811 - val_accuracy: 0.8515


In [16]:
test_pred = model.predict(test_input)

In [17]:
test_pred

array([[0.01665393],
       [0.6411139 ],
       [0.2945589 ],
       ...,
       [0.8700163 ],
       [0.9417828 ],
       [0.11258405]], dtype=float32)

In [18]:
test_id = test.id

In [19]:
test_id

0       13011
1       13722
2       21708
3        4989
4       15432
        ...  
5649      874
5650    19643
5651     8030
5652     9127
5653      254
Name: id, Length: 5654, dtype: int64

In [20]:
Predresult =pd.DataFrame()
Predresult['id']=test_id
Predresult['target'] = test_pred.round().astype(int)

In [21]:
Predresult

,id,target
0,13011,0
1,13722,1
2,21708,0
3,4989,0
4,15432,0
...,...,...
5649,874,0
5650,19643,0
5651,8030,1
5652,9127,1


In [22]:
Actualresult=pd.read_csv('../input/data-set/test_target.csv')

In [23]:
Actualresult

,id,target
0,13011,0
1,13722,0
2,21708,0
3,4989,0
4,15432,1
...,...,...
5649,874,0
5650,19643,0
5651,8030,1
5652,9127,0


In [24]:
from sklearn import metrics

In [25]:
metrics.accuracy_score(Actualresult.target,Predresult.target)

0.8638132295719845

In [26]:
metrics.roc_auc_score(Actualresult.target,Predresult.target)

0.7848573503578092

In [27]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Actualresult.target,Predresult.target)
print(cf_matrix)

[[3986  229]
 [ 541  898]]


In [28]:
Predresult.to_csv('csv_to_submit.csv', index = False)